In [ ]:
# Normal Comment
# ! Very Important Errors or things which must be fixed or be in attention immediately
# TODO: Things remaining to do
# ? Questions
# * Some Messages or Notes
# ^ Important Notes, Messages or things which may need attention in future
# & Important Commented out Code
# @ Section Starter
# ~ Run Time, PsuedoCode, Other Things

In [ ]:

#^###############################  Storage_Optimization.ipynb  ####################################
# ^ Author: Sukhendu Sain
# ^ Description: Main file of codebase. Houses main code
# ^ Data: 23-Nov-2024
#^################################################################################

In [ ]:
# Import Necessary Libraries, Utils, and Config Files
import utils
from config import *
import pandas as pd
import numpy as np
import math

# Data Import and Clean

In [ ]:

## @ Read FILE:: (AKINS FoMoCo_Piece_Sales_112222_YTD.xlsx) into Dataframe (Now Commented out)
# df_Akins = utils.read_excel(AKINS_FOMO_FILE_PATH)
# df_Akins['Part#'] = df_Akins['Part#'].apply(lambda a: "".join(str(a).split('-')))
# if print_df_after_import: utils.print_df(df_Akins, 200) # Print the Dataframe
# ~1-2secs

In [ ]:

## @ Read FILE:: (GPARTS Part Measures.xlsx) into Dataframe
df_Gparts = utils.read_excel(GPARTS_FILE_PATH)
if print_df_after_import: utils.print_df(df_Gparts) # Print the Dataframe
# ~23 secs

In [ ]:

## @ Read FILE:: (Wholesale JAN_Oct_Parts_Ranking_Counter_Invoices_All_Brands.xlsx) into Dataframe
df_Wholesale = utils.read_excel(WHOLESALE_FILE_PATH)

# Clean the Wholesale Dataframe
df_Wholesale['Description'] = df_Wholesale['Description'].astype(str)
df_Wholesale = df_Wholesale.drop(columns=[col for col in df_Wholesale.columns if 'Unnamed' in col], inplace=False)
df_Wholesale = df_Wholesale[(df_Wholesale['Vendor'] == 'FOR')].reset_index()
df_Wholesale.loc[df_Wholesale['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Avg. Cost'] = df_Wholesale['Description'].apply(lambda x: [i for i in x.strip().split("      ")][-1])
df_Wholesale.loc[df_Wholesale['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Description'] = df_Wholesale['Description'].apply(lambda x: "     ".join([i for i in x.strip().split("      ")][:-1]))

if print_df_after_import: utils.print_df(df_Wholesale) # Print the Dataframe
# ~7.5 secs

In [ ]:

## @ Read FILE:: (Service JAN_Oct_Parts_Ranking_ROs_All_Brands.xlsx) into Dataframe
df_Service = utils.read_excel(SERVICE_FILE_PATH)

# Clean the Service Dataframe
df_Service['Description'] = df_Service['Description'].astype(str)
df_Service = df_Service.drop(columns=[col for col in df_Service.columns if 'Unnamed' in col], inplace=False)
df_Service = df_Service[(df_Service['Vendor'] == 'FOR')].reset_index()
df_Service.loc[df_Service['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Avg. Cost'] = df_Service['Description'].apply(lambda x: [i for i in x.strip().split("      ")][-1])
df_Service.loc[df_Service['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Description'] = df_Service['Description'].apply(lambda x: "     ".join([i for i in x.strip().split("      ")][:-1]))
df_Service.loc[df_Service['Qty Sold'].apply(lambda x: len(str(x).split("      ")) > 1), 'Dollars Sold'] = df_Service['Qty Sold'].apply(lambda x: [i for i in str(x).strip().split("      ")][-1])
df_Service.loc[df_Service['Qty Sold'].apply(lambda x: len(str(x).split("      ")) > 1), 'Qty Sold'] = df_Service['Qty Sold'].apply(lambda x: "     ".join([i for i in str(x).strip().split("      ")][:-1]))

if print_df_after_import: utils.print_df(df_Service, 100) # Print the Dataframe
# ~4 secs

In [ ]:
## @ Read FILE:: (Counter Pad) into Dataframe

In [ ]:

## @ Read FILE:: (Availiable-Bin_Count) into Dataframe


## Data Processing & Calculation

In [ ]:

## @ Make a Big Final Dataframe

# * It will have the Columns - 'Part Number', 'Part Desc.', 'Active', 'Sold (Pcs.)', '0Dimensions', 'Length/Depth', 'Width', 'Height', 'Zone', 'Storage Type', 'Sub Storage', 'Number of Storage needed'
# It will have all the rows with common part nos. from 3 Files, having Appropriate Sold Pcs. Values, and Dimensions

main_list = [] # Initialize the New List, which will hold all rows before turning into DF
gParts_PartNos = set(df_Gparts['Svc Part Number']) # Get a Set of all Part Nos. of GParts

# Wholesale
common_part_numbers = gParts_PartNos & set(df_Wholesale['Part Number'])
df_PreMerge = df_Gparts.loc[df_Gparts['Svc Part Number'].isin(common_part_numbers), ['Svc Part Number', 'Svc Part Number Description', 'Is Active?', 'Prod Att - Length', 'Prod Att- Width', 'Prod Att - Height']]
for pn, pddesc, ac, dp, wd, ht in zip(df_PreMerge['Svc Part Number'], df_PreMerge['Svc Part Number Description'], df_PreMerge['Is Active?'], df_PreMerge['Prod Att - Length'], df_PreMerge['Prod Att- Width'], df_PreMerge['Prod Att - Height']):
    main_list.append([pn, pddesc, "", "Wholesale", ac, df_Wholesale[df_Wholesale['Part Number'] == pn]['Sold'].values[0], 0, 0, 0, 0, False, dp, wd, ht, "", "", "", "", 0, "", "", ""])

# Service
common_part_numbers = gParts_PartNos & set(df_Service['* indicates a superseded part\nPart Number'])
df_PreMerge = df_Gparts.loc[df_Gparts['Svc Part Number'].isin(common_part_numbers), ['Svc Part Number', 'Svc Part Number Description', 'Is Active?', 'Prod Att - Length', 'Prod Att- Width', 'Prod Att - Height']]
for pn, pddesc, ac, dp, wd, ht in zip(df_PreMerge['Svc Part Number'], df_PreMerge['Svc Part Number Description'], df_PreMerge['Is Active?'], df_PreMerge['Prod Att - Length'], df_PreMerge['Prod Att- Width'], df_PreMerge['Prod Att - Height']):
    main_list.append([pn, pddesc, "", "Service", ac, 0, df_Service[df_Service['* indicates a superseded part\nPart Number'] == pn]['Qty Sold'].values[0], 0, 0, 0, False, dp, wd, ht, "", "", "", "", 0, "", "", ""])


# Create the Main Dataframe
df_Main = pd.DataFrame(main_list)
df_Main.columns = ['Part#', 'Part Desc.', 'Part Category', 'DataSource', 'Active', 'Wholesale Sold', 'Service Sold', "Total Sold", 'OH Inventory', 'SKU Count', '0Dimensions', 'Depth', 'Width', 'Height', 'Zone', 'StorageType', 'SubStorage', 'Bin Type', 'Num. Bin Required', 'Actual Bin Type', 'Overflow Bins', 'Bin Location']

# @ Process & Clean the DF
# Merge the Parts present in both Service and Wholesale (Duplicates)
gbParts = df_Main.groupby('Part#').count()[df_Main.groupby('Part#').count()['Part Desc.'] == 2].index.to_list()
for pn in gbParts:
    df_Main.loc[(df_Main['Part#'] == pn) & (df_Main['DataSource'] == "Wholesale"), "Service Sold"] = df_Main.loc[(df_Main['Part#'] == pn) & (df_Main['DataSource'] == "Service"), "Service Sold"].values[0]
    df_Main = df_Main[(df_Main['Part#'] != pn) | (df_Main['DataSource'] == "Wholesale")]
# Set 0Dimensions
df_Main.loc[(df_Main["Depth"] == 0) | (df_Main["Height"] == 0) | (df_Main["Width"] == 0), "0Dimensions"] = True
# Drop 0Dimensions Rows if drop0Dims
if drop0Dims: df_Main = df_Main[df_Main["0Dimensions"] == False]
# Set Total_Sold
df_Main["Total Sold"] = df_Main["Wholesale Sold"].astype(int) + df_Main["Service Sold"].astype(int)
# Sort by 'Total Sold'
df_Main = df_Main.sort_values('Total Sold', ascending=False)
# ^ Add Random Values for OH Inventory temporarily
df_Main["OH Inventory"] = np.random.choice(np.arange(20), size=len(df_Main), replace=True)
# ^ Add Random Values for SKU Count temporarily
df_Main["SKU Count"] = np.random.choice(np.arange(20), size=len(df_Main), replace=True)

# Reset Index
df_Main = df_Main.reset_index(drop=True)

# ~4 min 28 Secs

In [ ]:
utils.print_df(df_Main)

In [ ]:

# ^ TESTING ---- Change some data Manually
df_Main.loc[df_Main["Part#"] == '6F2Z1A189A', ["Part Desc.","0Dimensions", "Depth", "Height", "Width", "OH Inventory"]] = ["6F2Z1A189A-TIRE",False, 28,28,3,100]
df_Main.loc[df_Main["Part#"] == '7L1Z1A189A', ["Part Desc.","0Dimensions", "Depth", "Height", "Width", "OH Inventory"]] = ["7L1Z1A189A-TIRE",False, 34,34,3.5,50]
df_Main.loc[df_Main["Part#"] == '9OO1183106436', ["0Dimensions", "Depth", "Height", "Width", "OH Inventory"]] = [False, 30,30,3,1000]
df_Main.loc[df_Main["Part#"] == '9OO439510', ["0Dimensions", "Depth", "Height", "Width", "OH Inventory"]] = [False, 45,45,4.5,500]
df_Main.loc[df_Main["Part#"] == '9OO1732002500', ["0Dimensions", "Depth", "Height", "Width", "OH Inventory"]] = [False, 50,50,5,250]
df_Main.loc[df_Main["Part#"] == '9OO3004901', ["0Dimensions", "Depth", "Height", "Width", "OH Inventory"]] = [False, 40,40,4,25]
# & df_Main[['TIRE' in s for s in df_Main["Part Desc."]]]

In [ ]:

## ^ Part Categorization  BUT Client Will Share Actual Part Category
def part_categorization(df_toBeCategorized, categoryColName):
    # TODO: Add more categories
    for i in range(df_toBeCategorized.shape[0]):
        desc = "-".join(df_toBeCategorized.loc[i, "Part Desc."].split("-")[1:])
        category = ""
        if "battery" in desc.lower():
            if desc.strip().lower() == "battery":
                category = "Battery"
            else:
                category = "Battery Accessory"
        elif "tire" in desc.lower():
            if desc.strip().lower() == "tire":
                category = "Tire"
            else:
                category = "Tire Accessory"            
        elif "hood" in desc.lower():
            if (desc.strip().lower() == "hood asy") | (desc.strip().lower() == "hood  asy"):
                category = "Hood"
            else:
                category = "Hood Accessory"
                
        elif desc.strip().lower() == "cover":
            category = "Bumper Cover"
        elif desc.strip().lower() == "seal":
            category = "Seal"
        elif desc.strip().lower() == "name plate":
            category = "Name Plate"
        elif desc.strip().lower() == "v-belt":
            category = "V-Belt"
        elif desc.strip().lower() == "v-belt":
            category = "V-Belt"
       
        elif ("blade" in desc.lower()) & ("wiper" in desc.lower()):
            category = "Wiper Blade"        
        elif ("arm" in desc.lower()) & ("wiper" in desc.lower()):
            category = "Wiper Arm"
        elif ("belt" in desc.lower()) & ("retractor" not in desc.lower()) & ("hole" not in desc.lower()) & ("cover" not in desc.lower()):
            category = "Belt"
        # elif ("hose" in desc.lower()) & ("vent" not in desc.lower()) & ("connect" not in desc.lower() & ("radiator" not in desc.lower()& ("heater" not in desc.lower()):
        #     category = "Hose"

        df_toBeCategorized.loc[i, categoryColName] = category
        

In [ ]:
part_categorization(df_Main, 'Part Category')
# ~4.7 secs

### Apply Zoning

In [ ]:

# @ Apply Zoning based on Time Period/Sale

## * Main Function for Apply Zoning
def Apply_Zoning(df_toBeZoned, zones, soldColName='Total Sold', zoneColName='Zone'): 
    df_toBeZoned.loc[:, zoneColName] = df_toBeZoned[soldColName].apply(lambda x: next((zone for zone, ratio in zones.items() if x >= ratio), list(zones.keys())[-1]))
    df_toBeZoned.loc[df_toBeZoned[soldColName] < 0, zoneColName] = None

In [ ]:

## * Run the Apply_Zoning on df_Main
Apply_Zoning(df_Main, zones, 'Total Sold', 'Zone')

In [ ]:
## * Check each Zone's number of Part Numbers
df_Main['Zone'].value_counts()

### Specialty Storage Assignment

#### Function for Bin Calculation

In [ ]:
def getNumOfBin(depth, width, height, raw_bin_dim, ohInven, fillFactor):
    # * Raw Bin Dimensions has this format :-  Height_Depth_Width
    if (raw_bin_dim != "") and (ohInven > 0):
        bin_height = float(raw_bin_dim.split("_")[1])
        bin_depth = float(raw_bin_dim.split("_")[2])
        bin_width = float(raw_bin_dim.split("_")[3])

        if raw_bin_dim.split("_")[0] == "BR":   # * Battery Rack
            return round((bin_depth / width), 4)      
               
        volBin = fillFactor * bin_height * bin_depth * bin_width    # * Available Storage Space
        volPart = height * depth * width
        if (volBin == 0):
            return 0
        
        numOfBins = round((ohInven * volPart) / volBin , 4)      # * Returns Fraction. 
        return numOfBins
    else:
        return 0

### Main Function for Storage Assignment

In [ ]:
def getStorage(zone, pcate, depth, width, height, ohInven, fillFactor):
    # Initialize the empty Variables
    storageType = ""
    subStorage = ""


    isSpec, storageType, subStorage, raw_bin_dim = utils.getSpecialtyStorage(pcate, depth, width, height)

    if not isSpec: 
        if (zone == "Red Hot") | (zone == "Red"):
            storageType, subStorage, raw_bin_dim = utils.getRedHotStorage(depth, width, height)
        elif (zone == "Orange") | (zone == "Yellow"):
            storageType, subStorage, raw_bin_dim = utils.getOrangeYellowStorage(depth, width, height)
        elif (zone == "Green") | (zone == "Blue"):   
            storageType, subStorage, raw_bin_dim = utils.getGreenBlueStorage(depth, width, height)
 
    numOfBins = getNumOfBin(depth, width, height, raw_bin_dim, ohInven, fillFactor)
    binDim = ""

    # * Build Bin Label with C (Clip), B (Bulk), D (Drawer), Battery Rack (BR), Tire Rack (TR) and Width-Depth-Height
    if raw_bin_dim.strip():   
        binDim =  raw_bin_dim.split('_')[0] + raw_bin_dim.split('_')[3] + raw_bin_dim.split('_')[2] + raw_bin_dim.split('_')[1]
    

    return storageType, subStorage, binDim, numOfBins # Return the Values

#### Apply the Storage Function

In [ ]:
for i in range(df_Main.shape[0]):
    # Set the Dimensions of the Data into Variables
    depth = df_Main.loc[i, "Depth"]
    height = df_Main.loc[i, "Height"]
    width = df_Main.loc[i, "Width"]

    zone = df_Main.loc[i, "Zone"]
    pcate = df_Main.loc[i, "Part Category"]
    ohInven = df_Main.loc[i, "OH Inventory"]

    # * If any dimension is zero, set empty Storage
    if df_Main.loc[i, "0Dimensions"] == True:
        df_Main.loc[i, "StorageType"] = ""
        df_Main.loc[i, "SubStorage"] = ""
        continue

    # Set Storage of the Parts
    df_Main.loc[i, "StorageType"], df_Main.loc[i, "SubStorage"], df_Main.loc[i, "Bin Type"], df_Main.loc[i, "Num. Bin Required"] = getStorage(zone, pcate, depth, width, height, ohInven, fillFactor)
# ~30 secs

In [ ]:

## @ HANGING Storage Calculation
# TODO: To Get the SKU Count for Hanging Storage
# * ASSUMPTION: Hook Length Based on SKU Count:
# *             6-inch hooks: For SKUs with 10 items or fewer
# *             12-inch hooks: For SKUs with 10–20 items

for hangingPN in df_Main.loc[df_Main['StorageType'] == 'Hanging Specialty Storage', "Part#"]: # Get Hanging Parts
    # & df_Main.loc[(df_Main['Part#'] == hangingPN), "Num. Bin Required"] = round(int(df_Main.loc[(df_Main['Part#'] == hangingPN), "OH Inventory"].values[0]) / hookDiv, 4)
    df_Main.loc[(df_Main['Part#'] == hangingPN), "SubStorage"], df_Main.loc[(df_Main['Part#'] == hangingPN), "Bin Type"], hookDiv = ("6-inch Hook", "HS06", 10) if df_Main.loc[(df_Main['Part#'] == hangingPN), "SKU Count"].values[0] <= 10 else ("12-inch Hook", "HS12", 20)
    df_Main.loc[(df_Main['Part#'] == hangingPN), "Num. Bin Required"] = int(df_Main.loc[(df_Main['Part#'] == hangingPN), "OH Inventory"].values[0])  # * Set No. of Hooks = Inventory Count

df_Main.loc[df_Main['StorageType'] != 'Hanging Specialty Storage', "SKU Count"] = 0
# ~7 Secs

In [ ]:

## @ Tire Storage Calculation
# TODO: FINALIZE  the Calculation of Tire Carousel Model Selection Based on Percentage 
# * ASSUMPTION:  Assign Carousel Model based on Diameter Group %age
# *   If 50% of Tires Have 33-inch or More Diameter, Assign Large-Storage (72-width carousel)
# *   ELSE,  For 28-inch or less, and,  between 28-33 inches, assign standard carrousel (48-width carousel) 

carousel_model = 'TR72' if df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'][df_Main['SubStorage'] == '33-inches Dia'].shape[0] / df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'].shape[0] >= tirePercent else 'TR48'
carousel_width = 72 if df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'][df_Main['SubStorage'] == '33-inches Dia'].shape[0] / df_Main[df_Main['StorageType'] == 'Tire Specialty Storage'].shape[0] >= tirePercent else 48
for tirePN in df_Main.loc[df_Main['StorageType'] == 'Tire Specialty Storage', "Part#"]:
    df_Main.loc[(df_Main['Part#'] == tirePN), "Num. Bin Required"] = round(int(df_Main.loc[(df_Main['Part#'] == tirePN), "OH Inventory"].values[0]) / (carousel_width // int(df_Main.loc[(df_Main['Part#'] == tirePN), "Width"].values[0])), 4)
    df_Main.loc[(df_Main['Part#'] == tirePN), "Bin Type"] = carousel_model
    

In [ ]:
utils.print_df(df_Main)

In [ ]:

# * Turn the Final Dataset into Excel
df_Main.to_excel('FinalDataset.xlsx', index=False) 
# ~8 Secs

In [122]:

# @ Do Actual Storage Allocation based on Inventory, Number of Bins availiable, Handle Overflow, etc...

df_binData = pd.DataFrame(columns=['Bin Label', 'Bin Category', 'Total Bins', 'Filled Amount', 'Bin Order', 'Bin Location', 'Availability Flag'])

# * High-Density Drawers (2)
binData = [ 
    {'Bin Label': 'D482406', 'Bin Category': 'Drawer', 'Total Bins': 5, 'Filled Amount': 3.5, 'Bin Order': 1, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
    {'Bin Label': 'D362406', 'Bin Category': 'Drawer', 'Total Bins': 4, 'Filled Amount': 0.8, 'Bin Order': 2, 'Bin Location': 'None', 'Availability Flag': 'Yes'}
 ]
# * Clip-Shelving (6)
binData.extend([
    {'Bin Label': 'C361215', 'Bin Category': 'Clip', 'Total Bins': 4, 'Filled Amount': 3.9, 'Bin Order': 3, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
    {'Bin Label': 'C361815', 'Bin Category': 'Clip', 'Total Bins': 6, 'Filled Amount': 2.0, 'Bin Order': 4, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
    {'Bin Label': 'C362415', 'Bin Category': 'Clip', 'Total Bins': 8, 'Filled Amount': 0.0, 'Bin Order': 5, 'Bin Location': 'None', 'Availability Flag': 'Yes'},    
    {'Bin Label': 'C481215', 'Bin Category': 'Clip', 'Total Bins': 3, 'Filled Amount': 0.0, 'Bin Order': 6, 'Bin Location': 'None', 'Availability Flag': 'Yes'},  
    {'Bin Label': 'C481815', 'Bin Category': 'Clip', 'Total Bins': 5, 'Filled Amount': 0.0, 'Bin Order': 7, 'Bin Location': 'None', 'Availability Flag': 'Yes'},   
    {'Bin Label': 'C482415', 'Bin Category': 'Clip', 'Total Bins': 7, 'Filled Amount': 0.0, 'Bin Order': 8, 'Bin Location': 'None', 'Availability Flag': 'Yes'}
])
# * Bulk-Storage ()
binData.extend([
  {'Bin Label': 'D482448', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 9, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
  {'Bin Label': 'D483648', 'Bin Category': 'Bulk', 'Total Bins': 5, 'Filled Amount': 0.0, 'Bin Order': 10, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
  {'Bin Label': 'D484248', 'Bin Category': 'Bulk', 'Total Bins': 6, 'Filled Amount': 0.0, 'Bin Order': 11, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
  {'Bin Label': 'D484848', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 12, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
  {'Bin Label': 'D487248', 'Bin Category': 'Bulk', 'Total Bins': 3, 'Filled Amount': 0.0, 'Bin Order': 13, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
  {'Bin Label': 'D489648', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 14, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
  {'Bin Label': 'D722448', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 15, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
  {'Bin Label': 'D723648', 'Bin Category': 'Bulk', 'Total Bins': 3, 'Filled Amount': 0.0, 'Bin Order': 16, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
  {'Bin Label': 'D724248', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 17, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
  {'Bin Label': 'D724848', 'Bin Category': 'Bulk', 'Total Bins': 6, 'Filled Amount': 0.0, 'Bin Order': 18, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
  {'Bin Label': 'D727248', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 19, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
  {'Bin Label': 'D729648', 'Bin Category': 'Bulk', 'Total Bins': 5, 'Filled Amount': 0.0, 'Bin Order': 20, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
  {'Bin Label': 'D962448', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 21, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
  {'Bin Label': 'D963648', 'Bin Category': 'Bulk', 'Total Bins': 5, 'Filled Amount': 0.0, 'Bin Order': 22, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
  {'Bin Label': 'D964248', 'Bin Category': 'Bulk', 'Total Bins': 7, 'Filled Amount': 0.0, 'Bin Order': 23, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
  {'Bin Label': 'D964848', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 24, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
  {'Bin Label': 'D967248', 'Bin Category': 'Bulk', 'Total Bins': 6, 'Filled Amount': 0.0, 'Bin Order': 25, 'Bin Location': 'None', 'Availability Flag': 'Yes'},
  {'Bin Label': 'D969648', 'Bin Category': 'Bulk', 'Total Bins': 4, 'Filled Amount': 0.0, 'Bin Order': 26, 'Bin Location': 'None', 'Availability Flag': 'Yes'}
])

# TODO: Add Specialty Storage!!!!

# Append the Data to the DF
df_binData = pd.concat([df_binData, pd.DataFrame(binData)], ignore_index=True)

utils.print_df(df_binData)

╒════╤═════════════╤════════════════╤══════════════╤═════════════════╤═════════════╤════════════════╤═════════════════════╕
│    │ Bin Label   │ Bin Category   │   Total Bins │   Filled Amount │   Bin Order │ Bin Location   │ Availability Flag   │
╞════╪═════════════╪════════════════╪══════════════╪═════════════════╪═════════════╪════════════════╪═════════════════════╡
│  0 │ D482406     │ Drawer         │            5 │             3.5 │           1 │ None           │ Yes                 │
├────┼─────────────┼────────────────┼──────────────┼─────────────────┼─────────────┼────────────────┼─────────────────────┤
│  1 │ D362406     │ Drawer         │            4 │             0.8 │           2 │ None           │ Yes                 │
├────┼─────────────┼────────────────┼──────────────┼─────────────────┼─────────────┼────────────────┼─────────────────────┤
│  2 │ C361215     │ Clip           │            4 │             3.9 │           3 │ None           │ Yes                 │
├────┼──

C:\Users\WINDOWS\AppData\Local\Temp\ipykernel_33088\4217100803.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_binData = pd.concat([df_binData, pd.DataFrame(binData)], ignore_index=True)


In [106]:

## ^ Psuedocode for main Logic
# ~ Start with 'Bin Type' as default assignment for the part --  ALLOCATE Bin As Per Availability, Zone, and Priority Rules
# ~ Add a column to 'BINS' dataframe, as Order or priority based on dimension for a given Rack Type (Drawer, Bulk, Clip, etc). 
# ~  This Order should only within specific Rack Type (Drawer, Bulk, Clip, etc). To change to Rack Type, use priorities given for 
# ~ Zones and Part Category. Always start with minimum dimension of the BIN
# ~ Start with 'Red Hot' Zone Parts to allocate Actual Bin. 
# ~ LOOP Over Zones. Then LOOP Over Parts. For Each Part, Check default 'Bin Type'. If Available for storage, then allocate it  
# ~ (Add the Bin Type in 'Actual Bin Allocation',  Increment the Fill_Quantity for the 'Bin Type' in second dataframe for BINS. 
# ~ Fill_Quantity can be fraction.  If Fill_Quantity is same  as  Available Bins, then Set the 'Available FLAG' for the BIN to 'NO'
# ~ That means, the BIN is fully occupied. Pick the next available BIN. 
# ~ Think About Overflow,  and Bin Location (Not sure if Bin Locations will be zone-wise)

In [107]:

# * Handle for Red Hot Zone First (without Specialty Storage)
for pn in df_Main.loc[df_Main['Zone'] == 'Red Hot', 'Part#']:
    # Get Variable
    actualBin =  ""
    overflowBin = ""

    partData = df_Main[df_Main['Part#'] == pn]
    partHeight = partData['Height'].values[0]
    partWidth = partData['Width'].values[0]
    partDepth = partData['Depth'].values[0]
    binType = partData['Bin Type'].values[0]
    storageType = partData['StorageType'].values[0]
    partOHInven = partData['OH Inventory'].values[0]

    # Base Continue Case
    if (storageType == "") or ("Specialty Storage" in storageType):
        continue

    binData = df_binData.loc[df_binData['Bin Label'] == binType]
    totalBinOfType = binData['Total Bins'].values[0]
    filledAmtOfBin = binData['Filled Amount'].values[0]
    binOrder = binData['Bin Order'].values[0]

    if binData['Availability Flag'].values[0]:      # If Bin availiable
        binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
        partVolume = partHeight * partWidth * partDepth 
        totalPartVolume = partOHInven * partVolume
        remainingVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 

        if (totalPartVolume <= remainingVolume):   # If all parts can fit within the remaining Bins of same label
            #df_Main.loc[df_Main['Part#'] == partData['Part#'].values[0], 'Actual Bin Allocation'] = binData['Bin Label']
            df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += float(totalPartVolume / binVolume)
            actualBin = f"{binType}({float(totalPartVolume / binVolume)},{partOHInven})"
            
            if (totalPartVolume == remainingVolume):  # If Bin fully used up, set Availability Flag to No
                df_binData.loc[df_binData['Bin Label'] == binType, 'Availability Flag'] = 'No'
        
        else:  # TODO: Overflow (Not all parts can fit here). Complete this
            overflowPartVolume = totalPartVolume
            df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'
            df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] = totalBinOfType
            actualBin = f"{binType}({float(totalBinOfType - filledAmtOfBin)},{remainingVolume/totalPartVolume})"
            
            for binType in df_binData.loc[df_binData['Bin Order'] > binOrder, 'Bin Label']:
                
                # Get Variables
                binData = df_binData.loc[df_binData['Bin Label'] == binType]
                totalBinOfType = binData['Total Bins'].values[0]
                filledAmtOfBin = binData['Filled Amount'].values[0]

                overflowPartVolume = overflowPartVolume - remainingVolume  # Calculate Overflow Volume
                binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
                remainingVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 

                overflowBin += f"|{binType}({float(totalPartVolume / remainingVolume)},{remainingVolume/totalPartVolume})"

                if (totalPartVolume <= remainingVolume):   # If all parts can fit within the remaining Bins of same label
                    # df_Main.loc[df_Main['Part#'] == partData['Part#'].values[0], 'Overflow Bins'] = binType
                    df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += float(totalPartVolume / binVolume)

                    if (totalPartVolume == remainingVolume):  # If Bin fully used up, set Availability Flag to No
                        df_binData.loc[df_binData['Bin Label'] == binType, 'Availability Flag'] = 'No'
                        break


           


    ## If the Bin is NOT available, Move to  Next Available Bin --  Code Here 
    #else:




C:\Users\WINDOWS\AppData\Local\Temp\ipykernel_33088\240905755.py:55: RuntimeWarning: divide by zero encountered in scalar divide
  overflowBin += f"|{binType}({float(totalPartVolume / remainingVolume)},{remainingVolume/totalPartVolume})"
C:\Users\WINDOWS\AppData\Local\Temp\ipykernel_33088\240905755.py:55: RuntimeWarning: divide by zero encountered in scalar divide
  overflowBin += f"|{binType}({float(totalPartVolume / remainingVolume)},{remainingVolume/totalPartVolume})"
C:\Users\WINDOWS\AppData\Local\Temp\ipykernel_33088\240905755.py:55: RuntimeWarning: divide by zero encountered in scalar divide
  overflowBin += f"|{binType}({float(totalPartVolume / remainingVolume)},{remainingVolume/totalPartVolume})"
C:\Users\WINDOWS\AppData\Local\Temp\ipykernel_33088\240905755.py:55: RuntimeWarning: divide by zero encountered in scalar divide
  overflowBin += f"|{binType}({float(totalPartVolume / remainingVolume)},{remainingVolume/totalPartVolume})"
C:\Users\WINDOWS\AppData\Local\Temp\ipykernel_33

In [123]:
pn = "XO5W20Q1SP"

# Get Variable
actualBin =  ""
overflowBin = ""

partData = df_Main[df_Main['Part#'] == pn]
partHeight = partData['Height'].values[0]
partWidth = partData['Width'].values[0]
partDepth = partData['Depth'].values[0]
binType = partData['Bin Type'].values[0]
storageType = partData['StorageType'].values[0]
partOHInven = partData['OH Inventory'].values[0]
partsLeft = partOHInven


binData = df_binData.loc[df_binData['Bin Label'] == binType]
totalBinOfType = binData['Total Bins'].values[0]
filledAmtOfBin = binData['Filled Amount'].values[0]
binOrder = binData['Bin Order'].values[0]

if binData['Availability Flag'].values[0]:      # If Bin availiable
    binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
    partVolume = partHeight * partWidth * partDepth 
    totalPartVolume = partOHInven * partVolume
    remainingVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 

    if (totalPartVolume <= remainingVolume):   # If all parts can fit within the remaining Bins of same label
        #df_Main.loc[df_Main['Part#'] == partData['Part#'].values[0], 'Actual Bin Allocation'] = binData['Bin Label']
        df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += float(totalPartVolume / binVolume)
        actualBin = f"{binType}({float(totalPartVolume / binVolume)},{partOHInven})"
        
        if (totalPartVolume == remainingVolume):  # If Bin fully used up, set Availability Flag to No
            df_binData.loc[df_binData['Bin Label'] == binType, 'Availability Flag'] = 'No'
    
    else:  # TODO: Overflow (Not all parts can fit here). Complete this
        overflowPartVolume = totalPartVolume
        df_binData.loc[df_binData['Bin Label'] == binType, 'Availiability Flag'] = 'No'
        df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] = totalBinOfType
        
        for binType in df_binData.loc[df_binData['Bin Order'] > binOrder, 'Bin Label']:
            
            # Get Variables
            binData = df_binData.loc[df_binData['Bin Label'] == binType]
            totalBinOfType = binData['Total Bins'].values[0]
            filledAmtOfBin = binData['Filled Amount'].values[0]

            overflowPartVolume = overflowPartVolume - remainingVolume  # Calculate Overflow Volume
            binVolume = fillFactor * (float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7]))
            remainingVolume = float(totalBinOfType - filledAmtOfBin) * binVolume  # Check for Remaining Vol in Bin 

            
            print(overflowBin, totalPartVolume, remainingVolume, binVolume, totalBinOfType, filledAmtOfBin, binType, overflowPartVolume)
            if (overflowPartVolume <= remainingVolume):   # If all parts can fit within the remaining Bins of same label
                # df_Main.loc[df_Main['Part#'] == partData['Part#'].values[0], 'Overflow Bins'] = binType
                df_binData.loc[df_binData['Bin Label'] == binType, 'Filled Amount'] += float(totalPartVolume / binVolume)
                overflowBin += f"|{binType}({float(totalPartVolume / remainingVolume)},{overflowPartVolume / partVolume})"

                if (overflowPartVolume == remainingVolume):  # If Bin fully used up, set Availability Flag to No
                    df_binData.loc[df_binData['Bin Label'] == binType, 'Availability Flag'] = 'No'
                break
            else:
                overflowBin += f"|{binType}({float(totalPartVolume / remainingVolume)},{remainingVolume/partVolume})"
            


        

# 30,202.1625
## If the Bin is NOT available, Move to  Next Available Bin --  Code Here 
#else:


actualBin, overflowBin

 30178.0264073175 27216.0 6804.0 6 2.0 C361815 29724.426407317496
|C361815(1.108834009675099,17.13511655866978) 30178.0264073175 72576.0 9072.0 8 0.0 C362415 2508.4264073174963


('',
 '|C361815(1.108834009675099,17.13511655866978)|C362415(0.41581275362816217,1.5792981653523874)')

In [109]:
binType = 'C361815'
float(binType[1:3]) * float(binType[3:5]) * float(binType[5:7])

9720.0